In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240715'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/3.model.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Make No'] = input_['Make No'].astype(int)
input_['Year'] = input_['Year'].astype(int)
input_['Model No'] = input_['Model No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        payload = {'jsn': {'tab': 'catalog',
                           'make': input_.loc[a, 'Make'],
                           'year': int(input_.loc[a, 'Year']),
                           'model': input_.loc[a, 'Model'],
                           'nodetype': 'model'}}

        data = {'func': 'navnode_fetch',
                'payload': json.dumps(payload),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['jsn']['tab'] == 'catalog' and json_['jsn']['make'] == input_.loc[a, 'Make'] and json_['jsn']['year'] == input_.loc[a, 'Year'] and json_['jsn']['model'] == input_.loc[a, 'Model'] and json_['jsn']['nodetype'] == 'model' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

            # = = = = = = = = = = = = = = =

            list_car_code = [json.loads(json_)['carcode'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            list_engine = [json.loads(json_)['engine'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            if not list_engine:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                                    'Make': input_.loc[a, 'Make'],
                                    'Year': input_.loc[a, 'Year'],
                                    'Model No': input_.loc[a, 'Model No'],
                                    'Model': input_.loc[a, 'Model'],
                                    'Engine No': [i+1 for i in range(len(list_engine))],
                                    'Engine': list_engine,
                                    'Car Code': list_car_code})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                                     'Make': input_.loc[a, 'Make'],
                                     'Year': input_.loc[a, 'Year'],
                                     'Model No': input_.loc[a, 'Model No'],
                                     'Model': input_.loc[a, 'Model']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']} > {input_.loc[a, 'Year']} > {input_.loc[a, 'Model No']}.{input_.loc[a, 'Model']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make No', 'Year', 'Model No', 'Engine No'],
                                                ascending=[True, False, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/4.engine-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make No', 'Year', 'Model No'],
                                            ascending=[True, False, True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/engine_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1986 > 12.VANAGON
[剩余数量：1000] - [当前时间：11:55:23]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1986 > 2.CABRIOLET
[剩余数量：999] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1986 > 9.QUANTUM
[剩余数量：998] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1986 > 11.TRANSPORTER
[剩余数量：997] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1985 > 4.COMBI
[剩余数量：996] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1985 > 5.CORSAR
[剩余数量：995] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1985 > 3.CARIBE
[剩余数量：994] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1986 > 4.COMBI
[剩余数量：993] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1984 > 1.ATLANTIC
[剩余数量：992] - [当前时间：11:55:24]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1985 > 10.SCIROCCO
[剩余数量：991] - [当前时间：11:55:25]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1985 > 12.VANAGON
[剩余数量：990] - [当前时间：11:55:25]

[状态：ok，尝试次数：1] - 258.VOLKSWAGEN > 1985 > 9.QUANTUM
[剩余数量：989] - [当前时间：11:55:25]

[状态：ok，尝试次数：1] - 258.